In [11]:
import os
import pickle
import numpy as np
import tensorflow.keras as keras
data_dir = "prepped_data/train_set/"

In [12]:
names = os.listdir(data_dir)


In [38]:
classes.shape

(2048, 12)

In [49]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [52]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', 
                 input_shape=(128, 128, 4)
                 
                ))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))              
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(12))

model.summary()

model.compile(loss=keras.losses.mean_squared_error,
              optimizer=Adam(lr=.001))




Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 126, 126, 32)      1184      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 12, 12, 128)      

In [ ]:
name = names[0]

def load_dataset(name):
    with open(data_dir + name, "rb") as f:
        dataset = pickle.load(f)

    data = []
    classes = []
    for elem in dataset:
        data_entry = np.array(np.stack(elem["data"], axis=-1), dtype=np.float32)
        data_entry += 1000
        data_entry /= 2000
        data.append(data_entry)

        class_entry = elem["classes"]
        class_entry = np.concatenate([class_entry[0] / 9, class_entry[1].flatten()])
        classes.append(class_entry)
    data = np.array(data)
    classes = np.array(classes)
    return data, classes


Epoch 1/5
2048/2048 [==============================] - 40s 20ms/sample - loss: 0.7970
Epoch 2/5
2048/2048 [==============================] - 29s 14ms/sample - loss: 0.3985
Epoch 3/5
2048/2048 [==============================] - 29s 14ms/sample - loss: 0.3504
Epoch 4/5
2048/2048 [==============================] - 37s 18ms/sample - loss: 0.3409
Epoch 5/5
1600/2048 [======================>.......] - ETA: 8s - loss: 0.3322 

In [ ]:
for _ in range(20):
    print("===========")
    for name in names:
        data, classes = load_dataset(name)
        loss_curve = model.fit(data, 
              classes,
              batch_size=200,
              epochs=5,
              verbose=1)
              #validation_data=(testdata, testclasses))